In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Input, Average
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os


In [2]:
train_dir = "../dataset/train"
val_dir   = "../dataset/val"

print("Train dir exists:", os.path.exists(train_dir))
print("Val dir exists:", os.path.exists(val_dir))


Train dir exists: True
Val dir exists: True


In [3]:
datagen = ImageDataGenerator(rescale=1./255)

train_data = datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical",
    shuffle=False
)

val_data = datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical",
    shuffle=False
)

num_classes = train_data.num_classes
print("Classes:", train_data.class_indices)


Found 1096 images belonging to 5 classes.
Found 1096 images belonging to 5 classes.
Classes: {'normal': 0, 'vitamin_A': 1, 'vitamin_B': 2, 'vitamin_C': 3, 'vitamin_D': 4}


In [4]:
cnn_model = load_model("../models/cnn_model.h5")
vgg16_model = load_model("../models/vgg16_model.h5")
resnet50_model = load_model("../models/resnet50_model.h5")
mobilenet_model = load_model("../models/mobilenet_model.h5")

print("All models loaded successfully")


All models loaded successfully


In [5]:
for model in [cnn_model, vgg16_model, resnet50_model, mobilenet_model]:
    model.trainable = False


In [6]:
input_layer = Input(shape=(224, 224, 3))

cnn_out = cnn_model(input_layer)
vgg_out = vgg16_model(input_layer)
resnet_out = resnet50_model(input_layer)
mobile_out = mobilenet_model(input_layer)

ensemble_output = Average()([
    cnn_out,
    vgg_out,
    resnet_out,
    mobile_out
])

ensemble_model = Model(inputs=input_layer, outputs=ensemble_output)


In [7]:
ensemble_model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

ensemble_model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 sequential_1 (Sequential)      (None, 5)            11169605    ['input_1[0][0]']                
                                                                                                  
 model_2 (Functional)           (None, 5)            21138757    ['input_1[0][0]']                
                                                                                                  
 model_1 (Functional)           (None, 5)            24113541    ['input_1[0][0]']            

In [8]:
history = ensemble_model.fit(
    train_data,
    validation_data=val_data,
    epochs=5
)


Epoch 1/5
35/35 [==============================] - 246s 7s/step - loss: 0.1602 - accuracy: 0.9973 - val_loss: 0.1431 - val_accuracy: 0.9982
Epoch 2/5
35/35 [==============================] - 236s 7s/step - loss: 0.1596 - accuracy: 0.9954 - val_loss: 0.1431 - val_accuracy: 0.9982
Epoch 3/5
35/35 [==============================] - 235s 7s/step - loss: 0.1581 - accuracy: 0.9954 - val_loss: 0.1431 - val_accuracy: 0.9982
Epoch 4/5
35/35 [==============================] - 233s 7s/step - loss: 0.1579 - accuracy: 0.9982 - val_loss: 0.1431 - val_accuracy: 0.9982
Epoch 5/5
35/35 [==============================] - 560s 16s/step - loss: 0.1567 - accuracy: 0.9973 - val_loss: 0.1431 - val_accuracy: 0.9982


In [9]:
ensemble_model.save("../models/ensemble_model.h5")
print("ensemble_model.h5 saved successfully")


ensemble_model.h5 saved successfully
